In [1]:
EXPERIMENT_INSTANCE_ID = "a636350c80144b038a0e805d8ba912a2"
N_CPUS = 8
TRACKING_URL = "file:///home/bogul/scalarizing/notebooks/mlruns"


In [2]:
import mlflow
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import Perceptron
from box import Box
from deslib import static
from deslib import des
from copy import deepcopy
from sklearn.metrics import accuracy_score

In [3]:
mlflow.set_tracking_uri(TRACKING_URL)


In [4]:
run = mlflow.start_run(run_id=EXPERIMENT_INSTANCE_ID)

In [5]:
params = Box(run.data.params,  box_recast={
                'bagging_size': int
            })

In [6]:
params.train_path = f"/home/bogul/scalarizing/notebooks/{params.train_path}"

In [7]:
params

<Box: {'bagging_size': 500, 'train_path': '/home/bogul/scalarizing/notebooks/../../datasets/processed/texture-train-2-s2.csv'}>

In [8]:
def read_dataset(path):
    data = pd.read_csv(path)
    x = data.drop('TARGET', axis=1).values
    y = data['TARGET'].values

    return {
        "x": x,
        "y": y
    }

In [9]:
train_path = params.train_path
test_path = train_path.replace('train','test')

In [10]:
dataset = Box({
        'train': read_dataset(train_path),
        'test': read_dataset(test_path),
        'name': train_path.split("/")[-1].replace("-train", '')
    })


# Experiments

In [11]:
class predict_wrapper(object):
    def __init__(self, predict_func, labels):
        self.predict_func = predict_func
        self.labels = labels

    def __call__(self, *args, **kwargs):
        return self.labels[self.predict_func(*args, **kwargs)]

def raise_not_implemented():
    raise NotImplemented("Predict proba is not supported")
def extract_classifiers_from_bagging(bagging):

    extracted = []
    for classifier in bagging.estimators_:
        cloned_classifier = deepcopy(classifier)
        cloned_classifier.predict = predict_wrapper(cloned_classifier.predict, bagging.classes_)
        cloned_classifier.predict_proba = raise_not_implemented

        extracted.append(cloned_classifier)

    return extracted

In [12]:
bagging = BaggingClassifier(base_estimator=Perceptron(), n_estimators=params.bagging_size, max_samples=0.3, random_state=42)
bagging.fit(dataset.train.x, dataset.train.y)

BaggingClassifier(base_estimator=Perceptron(), max_samples=0.3,
                  n_estimators=500, random_state=42)

In [13]:
bagging_test_acc = accuracy_score(bagging.predict(dataset.test.x), dataset.test.y)

In [14]:
from deslib.dcs.mcb import MCB
# Example of a dcs techniques
from deslib.dcs.ola import OLA
# Example of a des techniques
from deslib.des.des_p import DESP
from deslib.des.knora_e import KNORAE
from deslib.des.knora_u import KNORAU
from deslib.des.meta_des import METADES

In [15]:
knorau = KNORAU(extract_classifiers_from_bagging(bagging), k=5, random_state=42)
kne = KNORAE(extract_classifiers_from_bagging(bagging), k=5, random_state=42)
desp = DESP(extract_classifiers_from_bagging(bagging), k=5, random_state=42)
ola = OLA(extract_classifiers_from_bagging(bagging), k=5, random_state=42)
mcb = MCB(extract_classifiers_from_bagging(bagging), k=5, random_state=42)
# meta = METADES(extract_classifiers_from_bagging(bagging), k=5, random_state=42)

In [16]:
knorau.fit(dataset.train.x, dataset.train.y)
kne.fit(dataset.train.x, dataset.train.y)
desp.fit(dataset.train.x, dataset.train.y)
ola.fit(dataset.train.x, dataset.train.y)
mcb.fit(dataset.train.x, dataset.train.y)
# meta.fit(dataset.train.x, dataset.train.y)


MCB(k=5,
    pool_classifiers=[Perceptron(random_state=1952926171),
                      Perceptron(random_state=1761383086),
                      Perceptron(random_state=1449071958),
                      Perceptron(random_state=1910541088),
                      Perceptron(random_state=1341730541),
                      Perceptron(random_state=1286572245),
                      Perceptron(random_state=1005142668),
                      Perceptron(random_state=502852014),
                      Perceptron(random_state=186414760),
                      Pe...
                      Perceptron(random_state=420198453),
                      Perceptron(random_state=1930947788),
                      Perceptron(random_state=1419989711),
                      Perceptron(random_state=1242586904),
                      Perceptron(random_state=1836411051),
                      Perceptron(random_state=1419187828),
                      Perceptron(random_state=1418767096),
                      Perceptron(random_state=694812580),
                      Perceptron(random_state=1521681307),
                      Perceptron(random_state=649915709), ...],
    random_state=42)

In [17]:
knorau_acc = accuracy_score(knorau.predict(dataset.test.x), dataset.test.y)
kne_acc = accuracy_score(kne.predict(dataset.test.x), dataset.test.y)
desp_acc = accuracy_score(desp.predict(dataset.test.x), dataset.test.y)
ola_acc = accuracy_score(ola.predict(dataset.test.x), dataset.test.y)
mcb_acc = accuracy_score(mcb.predict(dataset.test.x), dataset.test.y)

In [18]:
mlflow.log_metrics({
    'knorau_test_acc': knorau_acc,
    'kne_test_acc': kne_acc,
    'desp_test_acc': desp_acc,
    'ola_test_acc': ola_acc,
    'mcb_test_acc': mcb_acc,
    'bagging_test_acc': bagging_test_acc,
    
})

In [19]:
mlflow.end_run()